# 1. 사용자 입력 받기
사용자로부터 필요한 정보를 입력받습니다. 입력값은 이후 매핑 및 데이터 필터링에 활용됩니다.


In [7]:
def get_user_input():
    """
    사용자로부터 입력값을 받습니다.
    """
    print("사용자 정보를 입력하세요:")
    seat_type = input("탑승 클래스 (Economy Class, Premium Economy, Business Class, First Class): ").strip()
    route = input("여행 구간 (예: Dubai to London): ").strip()
    analyze_reviews = input("리뷰 분석 여부 (Yes/No): ").strip().lower() == "yes"
    
    print("서비스 선호도를 입력하세요 (1~5 순위):")
    preferences = {
        "Seat Comfort": int(input("좌석 편안함 (1~5): ").strip()),
        "Cabin Staff Service": int(input("객실 승무원 서비스 (1~5): ").strip()),
        "Food & Beverages": int(input("기내 음식 및 음료 (1~5): ").strip()),
        "Inflight Entertainment": int(input("기내 엔터테인먼트 (1~5): ").strip()),
        "Ground Service": int(input("지상 직원 서비스 (1~5): ").strip()),
        "Wifi & Connectivity": int(input("기내 와이파이 (1~5): ").strip()),
        "Value For Money": int(input("가격 대비 가치 (1~5): ").strip()),
    }
    
    return {
        "seat_type": seat_type,
        "route": route,
        "analyze_reviews": analyze_reviews,
        "preferences": preferences,
    }

user_input = get_user_input()


사용자 정보를 입력하세요:
서비스 선호도를 입력하세요 (1~5 순위):


# 2. 데이터 로드 및 필요한 매핑 수행
JSON 파일에서 데이터를 로드하고 사용자 입력값에 필요한 IATA 코드만 매핑합니다.


In [8]:
import pandas as pd
import json

def load_airport_mapping(file_path):
    """
    OpenFlights 데이터셋에서 IATA 코드와 도시 이름 매핑 생성.
    """
    columns = ["Airport ID", "Name", "City", "Country", "IATA", "ICAO", "Latitude",
               "Longitude", "Altitude", "Timezone", "DST", "Tz database time zone", "Type", "Source"]
    airport_data = pd.read_csv(file_path, header=None, names=columns)
    return airport_data[["IATA", "City"]].dropna().set_index("IATA").to_dict()["City"]

file_path_airports = "/home/gyuha_lee/Opensource_dev/get_information/mine/airports.dat"
airport_city_mapping = load_airport_mapping(file_path_airports)

def preprocess_data(file_path, mapping):
    """
    데이터셋 로드 및 전처리.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    df = pd.DataFrame(data)

    # 필요한 열만 유지
    required_columns = [
        "Airline Name", "Route", "Seat Type", "Seat Comfort",
        "Cabin Staff Service", "Food & Beverages", "Inflight Entertainment",
        "Ground Service", "Wifi & Connectivity", "Value For Money", "Review Content"
    ]
    df = df[required_columns]
    return df

file_path_reviews = "/home/gyuha_lee/Opensource_dev/get_information/mine/all_airline_reviews.json"
df = preprocess_data(file_path_reviews, airport_city_mapping)


# 3. Route 정규화 및 데이터 필터링
동적으로 매핑된 데이터를 사용하여 Route 값을 정규화하고, 사용자 조건에 맞게 데이터를 필터링합니다.


In [ ]:
def normalize_route(route, mapping):
    """
    Route 값을 도시 이름 및 IATA 코드로 통일하여 정규화.
    """
    if pd.isnull(route) or isinstance(route, (int, float)):
        return ""
    if "via" in route.lower():
        route = route.split("via")[0].strip()
    route_parts = route.lower().replace("to", "").split()
    
    # IATA 코드 -> 도시 이름 변환
    normalized_parts = [mapping.get(part.upper(), part) for part in route_parts]

    # 두 개의 구간만 있는 경우 " to "로 연결
    if len(normalized_parts) == 2:
        return " to ".join(normalized_parts).strip()
    return ""

def normalize_input_route(input_route, mapping):
    """
    사용자 입력 Route를 정규화.
    """
    return normalize_route(input_route, mapping)

def match_route(input_route, data_route):
    """
    입력 Route와 데이터 Route를 양방향으로 비교.
    """
    try:
        input_start, input_end = input_route.split(" to ")
        data_start, data_end = data_route.split(" to ")
        return (
            (input_start == data_start and input_end == data_end) or
            (input_start == data_end and input_end == data_start)
        )
    except ValueError:
        return False

def filter_data(df, user_input, mapping):
    """
    사용자 입력값을 기반으로 데이터를 필터링.
    """
    seat_type = user_input["seat_type"].strip().lower()
    input_route = user_input["route"].strip()

    # 입력 Route 정규화
    normalized_input_route = normalize_input_route(input_route, mapping)

    # 데이터셋의 Route 정규화
    df["Normalized Route"] = df["Route"].apply(lambda r: normalize_route(r, mapping))

    # 유효한 Route만 필터링
    df = df[df["Normalized Route"] != ""]

    # 탑승 클래스 필터링
    filtered_df = df[df["Seat Type"].str.strip().str.lower() == seat_type]

    # 여행 구간 필터링
    filtered_df = filtered_df[filtered_df["Normalized Route"].apply(lambda r: match_route(normalized_input_route, r))]

    return filtered_df

filtered_data = filter_data(df, user_input, airport_city_mapping)

if filtered_data.empty:
    print("조건에 맞는 데이터가 없습니다. 입력 조건을 확인하세요.")
else:
    print("필터링된 데이터:")
    print(filtered_data.head(100))


필터링된 데이터:
                         Airline Name               Route      Seat Type  \
11452             Azerbaijan Airlines  DXB to LHR via GYD  Economy Class   
23895                        Gulf Air  DXB to LHR via BAH  Economy Class   
23896                        Gulf Air  DXB to LHR via BAH  Economy Class   
35833                Pegasus Airlines  STN to DXB via SAW  Economy Class   
35843                Pegasus Airlines  STN to DXB via SAW  Economy Class   
35845                Pegasus Airlines  STN to DXB via IST  Economy Class   
38314           Royal Brunei Airlines          DXB to LHR  Economy Class   
49387  Ukraine International Airlines  LHR to DXB via KBP  Economy Class   

       Seat Comfort  Cabin Staff Service  Food & Beverages  \
11452             3                    4                 3   
23895             3                    3                 0   
23896             1                    1                 1   
35833             5                    5                 

# 4. 리뷰 감성 분석
필터링된 데이터를 대상으로 BERT 모델을 사용하여 리뷰 텍스트 감성 점수를 분석합니다.


In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def analyze_review_sentiment_pytorch(df, analyze_reviews):
    """
    필터링된 데이터를 대상으로 리뷰 감성 분석 수행.
    """
    if not analyze_reviews:
        return df

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    model.to(device)

    def predict_sentiment(review):
        if pd.isnull(review) or not review.strip():
            return 0
        inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        return torch.argmax(probs, dim=-1).item() + 1

    total_reviews = len(df)
    print(f"총 리뷰 개수: {total_reviews}")

    sentiment_scores = []
    for idx, review in enumerate(df["Review Content"]):
        sentiment_scores.append(predict_sentiment(review))
        progress = (idx + 1) / total_reviews * 100
        print(f"\r진행률: {progress:.2f}% ({idx + 1}/{total_reviews})", end="")

    print("\n리뷰 감성 분석 완료!")
    df["Sentiment Score"] = sentiment_scores
    return df

filtered_data = analyze_review_sentiment_pytorch(filtered_data, user_input["analyze_reviews"])


총 리뷰 개수: 8
진행률: 100.00% (8/8)
리뷰 감성 분석 완료!


# 5. 추천 점수 계산 및 항공사 추천
사용자 선호도와 감성 분석 점수를 기반으로 추천 점수를 계산하고, 상위 항공사를 추천합니다.


In [11]:
def calculate_recommendation_score(df, user_input):
    preferences = user_input["preferences"]
    total_weight = sum(preferences.values())
    weights = {k: v / total_weight for k, v in preferences.items()}
    df["Recommendation Score"] = (
        df["Seat Comfort"] * weights["Seat Comfort"] +
        df["Cabin Staff Service"] * weights["Cabin Staff Service"] +
        df["Food & Beverages"] * weights["Food & Beverages"] +
        df["Inflight Entertainment"] * weights["Inflight Entertainment"] +
        df["Ground Service"] * weights["Ground Service"] +
        df["Wifi & Connectivity"] * weights["Wifi & Connectivity"] +
        df["Value For Money"] * weights["Value For Money"] +
        df["Sentiment Score"]
    )
    return df

def recommend_airlines(df):
    airline_scores = df.groupby("Airline Name")["Recommendation Score"].mean().sort_values(ascending=False)
    return airline_scores.head(5)

scored_data = calculate_recommendation_score(filtered_data, user_input)
recommendations = recommend_airlines(scored_data)

print("추천 항공사:")
print(recommendations)


추천 항공사:
Airline Name
Ukraine International Airlines    7.380952
Azerbaijan Airlines               5.238095
Royal Brunei Airlines             5.142857
Pegasus Airlines                  3.841270
Gulf Air                          2.166667
Name: Recommendation Score, dtype: float64
